# 세미 프로젝트 데이터 크롤링

## 1. 라이브러리 준비

In [1]:
import pandas as pd
import numpy as np

# 라이브러리 import
# 라이브러리 : 필요한 도구
from selenium import webdriver  # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC
import chromedriver_autoinstaller
from tqdm import tqdm
from tqdm import tqdm_notebook
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

## 2. 크롤링 참고

In [2]:
url = "https://ebook.pknu.ac.kr/ebook/?code=&mode=&sort=&cate_id=&page_num=1&view=50#list_tab"

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
driver.get(url)

In [3]:
# 책 종류 크롤링
overlays = "#booklist > ul > li:nth-child(1) > div.info > span > span"                                
kind = driver.find_element_by_css_selector(overlays)          
kind = kind.text
kind

'[자기관리]'

In [4]:
# 책 제목 크롤링
overlays = "#booklist > ul > li:nth-child(1) > div.info > span > a"                                 
tit = driver.find_element_by_css_selector(overlays)          
tit = tit.text
tit

'그릿 GRIT'

In [5]:
# 주소 크롤링
overlays = "#booklist > ul > li:nth-child(1) > div.info > span > a"                                 
add = driver.find_element_by_css_selector(overlays)          
add = add.get_attribute('href') 
print(add)

https://ebook.pknu.ac.kr/ebook/detail?goods_id=161100006


In [7]:
# 작가 크롤링
overlays = "#booklist > ul > li:nth-child(1) > div.info > ul > li:nth-child(1)"
writer = driver.find_element_by_css_selector(overlays)         
writer = writer.text
writer

'앤절라 더크워스'

In [8]:
# 렌탈 정보 크롤링
overlays = "#booklist > ul > li:nth-child(1) > div.info > div"                                 
rent = driver.find_elements_by_css_selector(overlays)    
rent = rent[0].text

## 3. 크롤링

### 1차 크롤링

In [3]:
page = 200 # 크롤링할 페이지 수 
add_list = [] # 주소 크롤링 데이터를 담을 그릇
chrome_path = chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
driver = webdriver.Chrome(chrome_path, chrome_options=options)

for j in tqdm_notebook(range(1,page+1)):
    
    dict = {}    # 전체 크롤링 데이터를 담을 그릇
        
    # 글 띄우기
    url = f"https://ebook.pknu.ac.kr/ebook/?code=&mode=&sort=&cate_id=&page_num={j}&view=50#list_tab"      
    driver.get(url)
    # 수집할 글 갯수 정하기
    number = 50
    
    # 수집한 url 돌면서 데이터 수집
    for i in range(1, number+1):

        # 크롤링
        # 예외 처리
        try : 
            target_info = {}  # 개별 내용을 담을 딕셔너리 생성

            # 책 종류 크롤링
            overlays = f"#booklist > ul > li:nth-child({i}) > div.info > span > span"      
            kind = driver.find_element_by_css_selector(overlays)          
            kind = kind.text

            # 책 제목 크롤링
            overlays = f"#booklist > ul > li:nth-child({i}) > div.info > span > a"                                 
            tit = driver.find_element_by_css_selector(overlays)          
            tit = tit.text

            # 주소 크롤링
            overlays = f"#booklist > ul > li:nth-child({i}) > div.info > span > a"                                 
            add = driver.find_element_by_css_selector(overlays)          
            add = add.get_attribute('href')
            add_list.append(add)

            # 작가 크롤링
            overlays = f"#booklist > ul > li:nth-child({i}) > div.info > ul > li:nth-child(1)"
            writer = driver.find_element_by_css_selector(overlays)         
            writer = writer.text

            # 렌탈 정보 크롤링
            overlays = f"#booklist > ul > li:nth-child({i}) > div.info > div"                                 
            rent = driver.find_elements_by_css_selector(overlays)    
            rent = rent[0].text                       

            # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
            target_info['kind'] = kind
            target_info['tit'] = tit
            target_info['writer'] = writer
            target_info['rent'] = rent

            # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
            dict[i] = target_info

            # 크롤링이 성공하면 글 제목을 출력하게 되고,
            print(i, tit)


        # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
        except:
            print("에러",i, tit)
            time.sleep(1)
            continue

    # 판다스 데이터프레임으로 만들기   
    globals()['result_df'+str(j)] = pd.DataFrame.from_dict(dict, 'index')
    

  0%|          | 0/200 [00:00<?, ?it/s]

1 그릿 GRIT
2 베로니카, 죽기로 결심하다
3 매우 예민한 사람들을 위한 책
4 작별하지 않는다
5 생각정리법
6 버터
7 타이탄의 도구들
8 여름의 빌라
9 자존감 수업
10 여행의 이유
11 지구에서 한아뿐
12 언어의 온도
13 바깥은 여름
14 상관없는 거 아닌가?
15 생각은 어떻게 행동이 되는가
16 살인자의 기억법
17 11문자 살인사건
18 그토록 가지고 싶은 문장들
19 나의 월급 독립 프로젝트
20 범죄, 그 심리를 말하다
21 우울할 땐 뇌과학
22 만화로 보는 99가지 발명품 이야기
23 허클베리 핀의 모험
24 10년 후 세계사 두 번째 미래
25 최소한의 선의
26 희랍어 시간
27 이기적 유전자
28 나는 나로 살기로 했다
29 쇼코의 미소
30 요즘 애들
31 도파민네이션
32 클라우스 슈밥의 제4차 산업혁명
33 하루 5분 탈무드 태교 동화
34 우울한 날엔 니체
35 아주 작은 습관의 힘
36 날마다 만우절
37 네이버 카페 플랫폼 비즈니스로 6개월 안에 연봉 벌기
38 결국 해내는 사람들의 원칙
39 오직 두 사람
40 유체역학
41 친일파는 살아있다
42 데일카네기 인간관계론
43 아가미
44 파과
45 나는 고양이로소이다
46 독서모임 꾸리는 법
47 1일 1페이지, 세상에서 가장 짧은 교양 수업 365
48 두 번째 지구는 없다
49 프로젝트 헤일메리
50 사회주의적 평화론
1 군주론
2 애써 말걸지 않아도 대화가 끊이지 않는 법
3 목소리를 드릴게요
4 밝은 밤
5 스몰토크: 대화가 쉬워지는 말의 공식
6 태양광발전 실험실습
7 너의 변명은 최고의 예술
8 [단독] 쉽게 행복해지는 사람
9 도시를 움직이는 모든 것들의 과학
10 168시간 일주일 사용법
11 30대에 다시 읽는 동화
12 20대라면 무조건 써먹는 경제상식
13 대학생의 착각
14 폭넓은 생각을 위한 역사 속 말빨 사전 101
15 아킬레우스의 노래
16 재무제표 모르면 주식투자 절대로 하지마라 (개정판)
17 숙명
18 하쿠바산장 살인사건
19 달

17 이런 얘기 하지 말까?
18 오늘 딱 하루만 잘 살아 볼까?
19 주식의 道(도)
20 세상에서 가장 행복한 100세 노인
21 한 게으른 시인의 이야기
22 일의 격
23 삶이 던지는 질문은 언제나 같다
24 합법적으로 세금 안 내는 110가지 방법 : 개인편
25 앞으로 10년 빅테크 수업
26 럭키 드로우
27 나의 MBTI가 궁금하단 마리몽
28 삼개주막 기담회 2
29 목요일 살인 클럽
30 완벽한 보고의 기술
31 성취예측모형
32 픽사 스토리텔링
33 공부란 무엇인가
34 나는 매일 블로그로 출근한다
35 우리가 만난 아이들
36 모방범 1
37 슈가레인 카페 디저트 클래스
38 서영동 이야기
39 대전환의 시대, 새로운 대한민국이 온다
40 랜선 인문학 여행
41 나를 알고 싶을 때 뇌과학을 공부합니다
42 질 건강 매뉴얼
43 재미로 읽다가 100점 맞는 색다른 물리학 (상편)
44 청춘, 여름, 꿈의 무대 고시엔
45 [단독] 대한민국 인구 트렌드 2022-2027
46 벤 호건 골프의 기본
47 페미돌로지
48 내 얼굴 비대칭?
49 실전 투자강의
50 나답게 일한다는 것
1 작고 단순한 삶에 진심입니다
2 다산의 마지막 질문
3 파이어 FIRE
4 시점의 힘
5 책으로 떠나는 여행
6 넥스트
7 아끼고 아낀 말
8 [단독] 행복을 파는 브랜드, 오롤리데이
9 30일 완독 책방
10 지금의 나로 충분하다
11 술꾼도시여자들 시즌1
12 월야환담 채월야 1
13 월야환담 채월야 2
14 월야환담 채월야 3
15 월야환담 채월야 4
16 월야환담 채월야 5
17 그 여자네 집
18 나비야 청산가자 1
19 나비야 청산가자 2
20 나는 프로그래머다
21 연금술사
22 조선의 여배우들
23 흐르는 강물처럼
24 익숙한 것과의 결별
25 세상에서 가장 유명한 50가지 이야기
26 비켜라 운명아, 내가 간다!
27 학사검전 2
28 글로벌 시민윤리
29 이상한 놈들이 온다
30 숲의 주인 1
31 진가도 2
32 진가도 3
33 진가도 4
34

18 태양의 탑 부록
19 내 인생의 만화책 - 캐릭터로 읽는 20세기 한국 만화사
20 태양광발전시스템 운영·유지보수 : 신재생에너지 4
21 유럽 문명의 아프리카 기원
22 집안일 반으로 줄이기
23 나는 내가 죽었다고 생각했습니다
24 고은 시선
25 지금, 비스마르크
26 나는 덜 일하기로 결심했다
27 변신
28 태도에 관하여(개정판)
29 탈코르셋: 도래한 상상
30 마흔, 부부가 함께 은퇴합니다
31 세상에서 가장 짧은 세계사
32 무엇이든 쓰게 된다
33 최고의 변화는 어디서 시작되는가
34 뉴파워: 새로운 권력의 탄생
35 매일 10분으로 끝장내는 기적의 영어회화패턴
36 꿈꾸는 다락방
37 내 문장이 그렇게 이상한가요?
38 모든 요일의 기록
39 이기적 감정 정리법
40 2억 빚을 진 내게 우주님이 가르쳐준 운이 풀리는 말버릇
41 마인드셋
42 심리학이 이렇게 쓸모 있을 줄이야
43 배낭여행자의 여행법
44 독학으로 합격하라
45 불안이라는 위안
46 애프터. 1: 치명적인 남자
47 애프터. 2: 이게 사랑일까
48 애프터. 3: 진실의 문
49 애프터. 4: 상처받은 영혼의 방
50 미국주식 처음공부
1 종교 권력은 세계 역사를 어떻게 움직였나
2 세상 친절한 세계사
3 그해, 여름 손님
4 단순함이 너의 모든 것을 바꾼다
5 대담한 작전
6 거울 앞에서 너무 많은 시간을 보냈다
7 하마터면 열심히 살 뻔했다(한정판 겨울 에디션)
8 나는 회사 다니면서 공부하기로 했다
9 아르테미스
10 재능의 불시착
11 딱 50일 압축 영문법
12 운을 부르는 습관
13 개미. 1
14 창문 넘어 도망친 100세 노인
15 고양이. 2
16 그래서 오늘 나는 외국어를 시작했다
17 일하지 않는 시간의 힘
18 일반화학(최신)
19 보기왕이 온다
20 잘생긴 개자식
21 쾌락독서
22 파이썬(Python) 200제
23 박시백의 조선왕조실록. 1 개국
24 박시백의 조선왕조실록. 14 숙종실록
25 조선에 온 서양 물건들
26 누구의 인정도 아닌
2

50 바람의 딸 걸어서 지구 세 바퀴 반 4
1 지도 밖으로 행군하라
2 바보 노무현
3 10억을 만든 사람들의 돈 IQ · EQ
4 20대 여자가 꼭 알아야 할 거절의 기술 34
5 30대 여자가 꼭 알아야 할 돈 관리법 43
6 30대가 꼭 알아야 할 돈 관리법 32
7 30대에 꼭 알아야 할 내집마련법 46
8 30대에 꼭 알아야 할 대화법 43
9 890만원으로 6억 만든 기적의 내집마련법
10 거절 못하는 나는 분명 문제가 있다
11 경리 · 회계 초보자가 꼭 알아야 할 23가지
12 공학박사 이창호의 부동산 시장을 보는 눈
13 쇼펜하우어의 토론의 법칙
14 제시 리버모어의 주식투자의 법칙
15 귀차르디니의 처세의 법칙
16 금융지식이 미래의 부를 결정한다
17 기획 초보자가 꼭 알아야 할 85가지
18 기획의 99%는 컨셉이다
19 나는 15억 벌어서 35세에 은퇴했다
20 나는 세상에서 가장 행복한 샐러리맨이다
21 나의 꿈 10억 만들기
22 난 은행적금보다 주식저축이 더 좋다
23 내 나이 스물다섯, 1년에 2억 번다
24 내 인생을 변화시키는 좋은 생각
25 논리적이면서도 매력적인 글쓰기의 기술
26 다름의 심리학 - 개정판
27 닥터아파트 CEO 오윤섭의 부동산 가치투자
28 단순하면서도 강력한 이동평균선 매매기법
29 당첨 확률을 10배 높이는 판교 청약의 기술
30 대한민국 소비 트렌드
31 돈 되는 펀드는 분명 따로 있다
32 돈 버는 방법은 분명 따로 있다
33 뛰어난 세일즈맨은 분명 따로 있다
34 뛰어난 직원은 분명 따로 있다
35 마케팅 실무자가 꼭 알아야 할 101가지
36 몸값을 최고로 올리는 자기경영의 기술
37 부동산 중개업자를 친구로 만들어라
38 부동산으로 10억 만들기
39 부자 기업 vs 가난한 기업
40 부자 되는데 60분이면 충분하다
41 부자들을 위한 변명
42 빈틈없는 자산관리를 위한 절세특강
43 사두기만 하면 오르는 대한민국 넘버원 아파트
44 서른 살부터 시작하는 부동산 재테크
45 서른 살부터 시작

...

20 대화의 수준을 높이는 레토릭 영어 명언
21 위험커뮤니케이션의 이론과 실제
22 우리 역사, 독도
23 한국사 기행
24 신HSK 5급 단어 핸드북
25 개미허리의 추억 - 우정편 (상)
26 개미허리의 추억 - 추억편 (하)
27 구보 박태원 소설 다시 읽기
28 한권으로 읽는 대한민국 대통령실록
29 김태희의 전략적 가사쓰기
30 소통, 진정성이 진정성을 만날 때
31 하상균의 파워 프레젠테이션
32 편세경의 실전강의 매뉴얼
33 한국어교육에서 한국 문화ㆍ문학 교육론
34 합법적으로 세금 안내는 110가지 방법 - 개인편 (2014년판)
35 열려라 아가리
36 오블라디 오블라다
37 로맨스 평행이론
38 넥스토피아 미래에 중독된 사람들
39 사람이 꽃이다
40 남자의 밥상
41 한식의 배신
42 살고 싶은 갖고 싶은 작은 집 작은 가구
43 단 한마디 말로도 박수 받는 힘
44 치유의 밥상
45 인생의 차이를 만드는 독서법 본깨적
46 신HSK 6급 단어 핸드북
47 그 남자의 새빨간 거짓말
48 스무 살, 버리지 말아야 할 것들
49 밤이 선생이다
50 지금 이 순간을 기억해 - 이주은의 벨 에포크 산책
1 데일 카네기 골든 메시지
2 이외수 김태원의 청춘을 위하여!
3 달콤한 악마가 내 안으로 들어왔다
4 한국인과 외국인을 위한 한국과 세계의 자원식물명 1
5 한국인과 외국인을 위한 한국과 세계의 자원식물명 2
6 약과 먹거리로 쓰이는 우리나라 자원식물
7 과학기술계 글쓰기
8 광통신시스템공학
9 역사 e 2
10 오늘도 최고의 날이 되십시오 - 미래를 여는 과학 편지
11 끝내기 영문법
12 인문학은 밥이다
13 구대검파 1
14 구대검파 2
15 사이킥위저드 1
16 사이킥위저드 2
17 사이킥위저드 3
18 사이킥위저드 4
19 사이킥위저드 5
20 사이킥위저드 6
21 사이킥위저드 7
22 사이킥위저드 8
23 사이킥위저드 9
24 사이킥위저드 10
25 사이킥위저드 11 (완결)
26 퀀텀스토리
27 체험으로 읽는 티벳 사자의 서
28 

12 윤기정_단편집 4_(사생아_적멸)
13 윤기정_단편집 5_(아씨와 안잠이_공사장)
14 윤기정_단편집 6_(어머니와 아들_차부)
15 윤기정_단편집 7_(천재_리창섭 브리가다)
16 윤동주_단편집 1_(달을 쏘다_화원에 꽃이 핀다)
17 윤백남_단편집 1_(장마가 실어온 발복_후백제비화_사각전기)
18 윤백남_단편집 2_(안류정_경벌포의_괴승신수)
19 윤백남_단편집 3_(상방기현-종침교명유래기_소설정획점고인)
20 윤백남_단편집 4_(이식과 도승_우연의 기적_적괴유이)
21 윤백남_단편집 5_(정열의 낙랑공주_순정의 호동왕자_집념)
22 윤백남_단편집 6_(초췌연화편_투환금은_원수로은인)
23 윤백남_단편집 7_(홍윤성과 절부_보은단 유래)
24 이갑용_단편집 1_(B녀의 소묘_O형의 인간_굉장씨)
25 이갑용_단편집 2_(그 전날 밤_기우제_기차와 박노인)
26 이갑용_단편집 3_(나는 보아 잘 안다_나랏님 전 상사리_노래를 잊은 사람)
27 이갑용_단편집 4_(농부전초_누이의 집_며느리)
28 이갑용_단편집 5_(두훈시_또 하나의 위선_만보노인)
29 이갑용_단편집 6_(모우지도_산가_문서방)
30 이갑용_단편집 7_(벽화_사위_이단자)
31 이갑용_단편집 8_(소녀_실제기_아침)
32 이갑용_단편집 9_(안달소전_연사봉_우심)
33 이갑용_단편집10_(원균의 후예_유모_전기)
34 이광수_단편집 1_(H군을 생각하고_모르는 여인_거룩한 이의 죽음)
35 이광수_단편집 2_(길놀이_소년의 비애_난제오)
36 이광수_단편집 3_(꿈_무명씨전_방황)
37 이광수_단편집 4_(사랑에 주렸던 이들_옥수수_윤광호)
38 이광수_단편집 5_(할멈_혼인_가실)
39 이광수_단편집 6_(무명)
40 이광수_단편집 7_(어린 벗에게)
41 이녕
42 이명선_단편집 1_(개시대길_복수_고향)
43 이명선_단편집 2_(노신선생부인 경송여사_닭)
44 이명선_단편집 3_(일선문 춘향전_춘향적 해석_이춘풍전의 현대화)
45 이명선_단편집 4_(빵떡)
46 이명선_단편집 5

50 마르케스
1 말하다
2 대한민국 청년 일자리 프로젝트
3 결론부터 써라
4 고객님, 여기서 이러시면 안 됩니다
5 용감한 친구들 1
6 용감한 친구들 2
7 한국 사회 빈부의식은 어떻게 변했는가 - 살림지식총서 509
8 너는 나에게 상처를 줄 수 없다 2
9 글로벌 식품안전 동향보고서
10 인플루엔자 백신 헤마글루티닌(HA) 함량시헙법
11 임상통계 사례집
12 디지털 방송서비스 이용활용화 방안 연구
13 원자력 백서 (2014)
14 질의회신 사례방 - 도시 및 주거환경정비법
15 색다른 중국유학 체험기
16 사람의 마음을 읽는 시간 0.2초
17 관계를 회복하는 용기
18 질투의 민낯
19 9일의 묘
20 미생 드라마 포토 에세이
21 필요한 사람인가
22 하하의 썸싱
23 눈의 경전
24 마도십병 1
25 마도십병 2
26 마도십병 3
27 마도십병 4
28 마도십병 5
29 마도십병 6
30 마도십병 7
31 마도십병 8 (완결)
32 블랙이글스에게 배워라!
33 나는 작가가 되기로 했다
34 매혹의 러시아로 떠난 네 남자의 트래블로그, 러시아 여행자 클럽
35 두뇌와의 대화 : 하버드 의대 교수 앨런 로퍼의
36 위(Oui), 셰프
37 스포츠 영어회화
38 Simple 심플
39 2013 공무원 시험제도 개편 합격 가이드 - 박문각이 답이다
40 경찰기출 완전정복 경찰학개론
41 경찰기출 완전정복 형법
42 경찰기출 완전정복 형사소송법
43 2014 공사 공단 기출문제집 일반상식
44 2013 공무원 시험제도 개편 합격 가이드 - 남부를 켜면 합격이 보인다
45 2014 KT종합인적성검사 7일 벼락치기 실전모의고사
46 2014 CJ CAT&CJAT CJ 종합적성검사 7일 벼락치기 실전모의고사
47 2013 공무원 기출 완전정복 영어
48 2013 공무원 기출 완전정복 한국사
49 2015 공사공단 인적성검사
50 2013 9급 국가직 기출해설집
1 spa 일반상식
2 공무원! 면접의 정석
3 2013 9급 지방직 기출해설집
4 2016 기아

7 한겨울에 녹다
8 파리의 우울
9 궁금증이 지식이 되는, 아하!
10 나답게 살 용기
11 나 요즘, 분노조절 장애인가?
12 전략 천재가 된 홍대리
13 우리는 부끄러운 청춘으로 살 수 없다
14 처음 베이징에 가는 사람이 가장 알고 싶은 것들
15 심리학자와 함께 가는 치유의 영화관
16 신의 위대한 질문
17 인간의 위대한 질문
18 고양이의 서재
19 스타트업 바이블
20 페루, 내 영혼에 바람이 분다
21 독
22 자기 앞의 생
23 삶을 바꾸려면 음식을 바꿔라
24 가끔은 격하게 외로워야 한다
25 마흔, 논어를 읽어야 할 시간 2
26 Decoding the TOEFL iBT Actual Test Reading 1
27 Decoding the TOEFL iBT Actual Test READING 2
28 Decoding the TOEFL iBT Actual Test WRITING 1
29 Decoding the TOEFL iBT Actual Test WRITING 2
30 다이나믹 일본어 독해 중급
31 다이나믹 일본어 독해 초급
32 야사시이 일본어 문법
33 일본어 문법 한권으로 끝내기
34 일본어 상용한자 2136 한권으로 끝내기
35 잃어버린 희망(HOPELESS)
36 박철범의 하루공부법 2 (개정판)
37 박철범의 방학공부법
38 나는 단순하게 살기로 했다
39 부자가 되는 정리의 힘
40 신경 쓰지 않는 연습
41 그림을 품은 한자
42 위대한 영화감독들 A To Z
43 지구물리탐사 개론
44 한글 2007 : My Love 31
45 전쟁과 사랑 제2권
46 My First TOEIC L/C 2 - 19강
47 신관광 사업론
48 종횡만리 제2권
49 골짜기
50 내 마음의 벽
1 나인 드래곤
2 역사 e 4
3 위대한 전환
4 Nicomachean Ethics
5 틀을 깬 과학자들 (개정판)
6 한 가지 생각
7 비주얼 경제사
8 개정판 영어 매일 확인학습: 9,7급 공무원 시험대비
9 개정판 한국사 매일 확인학습: 9,7급 

49 마스터 알고리즘
50 모든 요일의 여행
1 수고했어, 오늘도
2 꾸짖는 기술
3 담배를 든 루스
4 어젯밤 꿈이 나에게 말해주는 것들
5 이제는 이기는 인생을 살고 싶다
6 처음 시작하는 미술치료
7 처음 시작하는 심리검사와 심리평가
8 삶과 문명의 눈부신 비전 열하일기
9 필사의 기초
10 맹자를 읽다
11 떠날 것인가, 남을 것인가
12 내 인생을 바꾸는 시간관리 자아실현
13 디어 마이 프렌즈 1 (노희경 원작 소설)
14 문제해결자
15 화내지 않고 가르치는 기술
16 블랙 오로라
17 이중섭, 떠돌이 소의 꿈
18 아침식사의 문화사 Breakfast
19 코타로와 나
20 이룰 수 없는 꿈을 꾸고 저 하늘의 별을 잡자
21 다시 만나자 우리
22 정말 재미있는 심리학 콘서트
23 딱 10일 동안 아이슬란드
24 이탈리아 남부 기행
25 유럽, 가지 않을 이유가 없었다
26 프랑스 대통령의 모자
27 빨간 수첩의 여자
28 개와 꽃과 친구가 있는 날
29 나는 왜 싫다는 말을 못 할까 - 삶이 심플해지는 거절의 힘
30 익스팬스 : 깨어난 괴물 1
31 익스팬스 : 깨어난 괴물 2
32 내 손 놓지 마
33 다시 시작하는 독서
34 순수의 시대 The Age of Innocence (영어 원서 읽기)
35 유피디의 독일의 발견
36 오스트리아 홀리데이
37 런던 홀리데이
38 앙코르와트 홀리데이
39 여름, 어디선가 시체가
40 시 읽는 CEO, 처음 시작하는 이에게
41 심연
42 콜 더 미드와이프
43 세상에서 가장 재미있는 심리학 백과사전 2
44 매혹하는 식물의 뇌
45 그리스 신화밖에 모르는 당신에게
46 끝에서 두 번째 사랑
47 범죄의 탄생
48 내가 미래를 앞서가는 이유
49 마케팅을 부탁해
50 빨강머리 앤이 하는 말
1 리더의 아침수업
2 다중격차 한국 사회의 불평등 구조
3 한국의 불평등 2016
4 연애소설 읽는 노인
5 하루 1분 추리게임
6 레퀴엠
7 어쩌다 이런 가족
8 화이트 나이트
9 인생교과서 미켈란젤로
10 아웃사

In [4]:
df_list = []
for i in range(page):
    df_list.append(globals()['result_df'+str(i+1)])

In [5]:
df_list

[        kind                            tit                  writer  \
 1     [자기관리]                        그릿 GRIT                앤절라 더크워스   
 2       [문학]                 베로니카, 죽기로 결심하다                 파울로 코엘료   
 3    [인문／사회]               매우 예민한 사람들을 위한 책                     전홍진   
 4       [문학]                       작별하지 않는다                      한강   
 5     [자기관리]                          생각정리법                  아카바 유지   
 6       [문학]                             버터     <유즈키 아사코> 저/<권남희> 역   
 7     [자기관리]                       타이탄의 도구들                   팀 페리스   
 8       [문학]                         여름의 빌라                     백수린   
 9   [국어와외국어]                         자존감 수업                     윤홍균   
 10      [문학]                         여행의 이유                     김영하   
 11      [문학]                       지구에서 한아뿐                     정세랑   
 12      [문학]                         언어의 온도                     이기주   
 13      [문학]                         바깥은 여름                    

In [6]:
# 데이터 프레임 합치기 수동 조작 필요 50페이지 당 1개 데이터 프레임 존재
result_df = pd.concat(df_list, ignore_index=True)

In [7]:
len(result_df)

10000

In [8]:
result_df

,kind,tit,writer,rent
0,[자기관리],그릿 GRIT,앤절라 더크워스,"보유 5, 대출 2, 예약 0, 누적대출 37, 누적예약 1"
1,[문학],"베로니카, 죽기로 결심하다",파울로 코엘료,"보유 5, 대출 3, 예약 0, 누적대출 28, 누적예약 1"
2,[인문／사회],매우 예민한 사람들을 위한 책,전홍진,"보유 5, 대출 5, 예약 1, 누적대출 28, 누적예약 6"
3,[문학],작별하지 않는다,한강,"보유 5, 대출 3, 예약 0, 누적대출 27, 누적예약 0"
4,[자기관리],생각정리법,아카바 유지,"보유 5, 대출 3, 예약 0, 누적대출 25, 누적예약 5"
...,...,...,...,...
9995,[문학],고무보트를 타고 상어 잡는 법,모르텐 스트뢰크스네스,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0"
9996,[자기관리],지지 않는 대화,다카하시 켄타로,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0"
9997,[문학],헤밍웨이 죽이기,엘러리 퀸,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0"
9998,[자기관리],단순하게 생각하는 연습,구사나기 류슌,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0"


In [41]:
# 저장하기
result_df.to_csv("세미 프로젝트 책추천 크롤링(내용제외).csv", encoding='utf-8-sig')

In [43]:
import csv
with open('세미 프로젝트 책추천 크롤링(주소).csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(add_list)

### 2차 크롤링( 책 내용 수집)

In [29]:
add_list[388]

'https://ebook.pknu.ac.kr/ebook/detail?goods_id=N1507283'

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [11]:
body_list = []
chrome_path = chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument('--no-sandbox') # 크롬이 안 열릴때, 크롬 보안을 해제하는 기능
options.add_argument('--disable-dev-shm-usage') # 공유 메모리를 담당하는 디렉토리를 사용하지 않는 다는 의미
driver = webdriver.Chrome(chrome_path, chrome_options=options)
for i in tqdm_notebook(add_list):

    try :
        #내용 크롤링
        driver.get(i)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#bookinfo"))) # 페이지 로딩을 특정조건(css 셀렉터)이 로딩될때까지 대기
        overlays = "#bookinfo"                                 
        body = driver.find_element_by_css_selector(overlays)          
        body = body.text
        body_list.append(body)
        
    except Exception as e:
        print("에러",i)
        body_list.append('error') # 리스트 append를 null 값을 넣을 수 없기에 임의의 문자를 입력
        time.sleep(2)
        driver = webdriver.Chrome(chrome_path, chrome_options=options) # 드라이버가 오류나면 종료가 되기에 새로운 드라이버 작동
        driver.implicitly_wait(30) # 드라이버 작동하고 로딩될때까지 대기
        continue

  0%|          | 0/10000 [00:00<?, ?it/s]

In [12]:
len(body_list)

10000

In [13]:
# 드라이버 완전 종료
driver.quit()

### 크롤링 데이터 합치기

In [14]:
# 리스트 -> 시리즈 변환
result_df_body = pd.Series(body_list) 


# 1차 크롤링 + 2차 크롤링 합치기
result_df = pd.concat([result_df,result_df_body],axis=1)



In [15]:
result_df

,kind,tit,writer,rent,0
0,[자기관리],그릿 GRIT,앤절라 더크워스,"보유 5, 대출 2, 예약 0, 누적대출 37, 누적예약 1","책소개\n*** TED 조회 수 1,000만 돌파! *** 전세계 25개국 동시 출..."
1,[문학],"베로니카, 죽기로 결심하다",파울로 코엘료,"보유 5, 대출 3, 예약 0, 누적대출 28, 누적예약 1","책소개\n세계적 밀리언셀러 작가, '소설의 연금술사'로 불리는 파울로 코엘료의 신작..."
2,[인문／사회],매우 예민한 사람들을 위한 책,전홍진,"보유 5, 대출 5, 예약 1, 누적대출 28, 누적예약 6",책소개\n예민성은 어떻게 조절될 수 있을까\n\n저자는 이 책에 나오는 사람마다 상...
3,[문학],작별하지 않는다,한강,"보유 5, 대출 3, 예약 0, 누적대출 27, 누적예약 0",책소개\n무엇을 생각하면 견딜 수 있나.\n가슴에 활활 일어나는 불이 없다면.\n기...
4,[자기관리],생각정리법,아카바 유지,"보유 5, 대출 3, 예약 0, 누적대출 25, 누적예약 5","책소개\n“망설이지 말고, 당황하지 말고\n0.1초 안에 생각해서 당당하게 반박하라..."
...,...,...,...,...,...
9995,[문학],고무보트를 타고 상어 잡는 법,모르텐 스트뢰크스네스,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",책소개\n“헤밍웨이의 《노인과 바다》 같은 고전이 될 만하다!”\n\n_〈베르겐스 ...
9996,[자기관리],지지 않는 대화,다카하시 켄타로,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0","책소개\n2,500년 동안 세계를 움직여온 무적의 대화법!\n\n아리스토텔레스의 《..."
9997,[문학],헤밍웨이 죽이기,엘러리 퀸,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",책소개\n“\n이보다 더 걸출한 작가들의 선집을 상상하기란 불가능하다!\n12명의 ...
9998,[자기관리],단순하게 생각하는 연습,구사나기 류슌,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0","책소개\n《반응하지 않는 연습》의 저자,\n구사나기 류슌이 소개하는\n내 마음 정리..."


### 데이터 전처리

In [56]:
# 글자수 확인하기
result_df[0].str.len()

0       6264.0
1       3904.0
2       3152.0
3       1249.0
4       5444.0
         ...  
9995       NaN
9996       NaN
9997       NaN
9998       NaN
9999       NaN
Name: 0, Length: 10000, dtype: float64

In [57]:
# 적정한 글자수 확인 
# 최소 400자 이상이어야 3줄 이상의 책 요약이 가능
result_df[0].loc[996]

'책소개\n저는 달을 내리는 일을 합니다.\n\n“사람은 저마다의 달을 가지고 있죠. 사실 우리가 보는 달이 하나라고 생각할 수 있지만, 제가 보는 달과 당신이 보는 달은 다릅니다.” - 책 속에서\n\n커다란 달이 떠오르고, 어느 골목에 ‘카페 문 CAFE MOON’이 열린다.\n퇴사와 미래에 대해 고민하며 길을 걷던 정아는 커다란 달이 자신에게 다가오는 듯 가까워지는 것을 느끼고, 달을 피해 숨어든 골목에서 CAFE MOON을 발견한다. 이끌리듯 들어간 CAFE MOON은 단 한 가지 메뉴만을 판매한다. 바로 ‘달 라떼’다.\n바리스타 문은 자연스럽게 정아에게 달 라떼를 권하고, 그녀의 고민 한 조각과 바리스타 문의 달 조각이 합쳐져 만들어진 달 라떼를 마신 정아는 개운한 기분을 느끼며 정체 모를 자신감을 느낀다. 그녀는 바리스타 문에게 ‘달 라떼’ 이야기를 들으며 묘한 끌림을 느끼고, 커다란 그 달로 인해 각 지역에 흩어져 있는 ‘비슷한 존재들’이 CAFE MOON으로 모이기 시작한다.\n과연, 그들의 정체는 무엇이고 ‘CAFE MOON’의 달 라떼에는 어떤 신비한 능력이 있는 걸까?\n저자소개\n저자 : 김성진\n김성진 네이버 오디오 클립 ‘김성진의 읽어주는 남자’ 진행 및 전업 작가. 밥을 짓듯이 글을 짓고 싶은, 그래서 글을 읽어주시는 분들의 마음을 살찌워 드리고 싶은 작가입니다. 제 글이 편지를 다시 쓰는 씨앗이 되어 마음에 심어지길 바랍니다.전작 《컬러, 옐로우》, 《컬러 레드》 (ebook)\n목차\n프롤로그. 달을 켜고 끄는 것\n\n1. CAFE MOON\n2. 형광색 표정\n3. 라떼는 말이죠\n4. 마스터 프라이데이의 독특한 취미\n5. 꿈을 되감는 방법\n6. 고민 들어주는 남자\n7. 비슷한 ‘존재’들\n8. 쇼 미 더 머니\n9. 다시 한번 소녀가 되는 방법\n10. 붉은여우\n11. 피스타치오 플랫 화이트\n12. 마술 좋아하세요?\n13. 고민을 해결하는 가장 좋은 방법\n14. 살아 있는 지구\n15. 토끼보다 더 사람에 가까

In [18]:
# 400자 이하인 인덱스 값
result_df[result_df[0].str.len() <= 400].index

Int64Index([  21,   39,   49,   55,   62,   76,  102,  103,  153,  156,
            ...
            9729, 9730, 9731, 9732, 9733, 9734, 9763, 9928, 9931, 9960],
           dtype='int64', length=1380)

In [19]:
# 400자 이하인 인덱스 값 삭제
result_df.drop(result_df[result_df[0].str.len() <= 400].index, axis = 0, inplace = True )

In [20]:
# 인덱스 값 설정
result_df.reset_index(inplace=True)
result_df.drop(['index'], axis = 1, inplace = True)

In [21]:
result_df

,kind,tit,writer,rent,0
0,[자기관리],그릿 GRIT,앤절라 더크워스,"보유 5, 대출 2, 예약 0, 누적대출 37, 누적예약 1","책소개\n*** TED 조회 수 1,000만 돌파! *** 전세계 25개국 동시 출..."
1,[문학],"베로니카, 죽기로 결심하다",파울로 코엘료,"보유 5, 대출 3, 예약 0, 누적대출 28, 누적예약 1","책소개\n세계적 밀리언셀러 작가, '소설의 연금술사'로 불리는 파울로 코엘료의 신작..."
2,[인문／사회],매우 예민한 사람들을 위한 책,전홍진,"보유 5, 대출 5, 예약 1, 누적대출 28, 누적예약 6",책소개\n예민성은 어떻게 조절될 수 있을까\n\n저자는 이 책에 나오는 사람마다 상...
3,[문학],작별하지 않는다,한강,"보유 5, 대출 3, 예약 0, 누적대출 27, 누적예약 0",책소개\n무엇을 생각하면 견딜 수 있나.\n가슴에 활활 일어나는 불이 없다면.\n기...
4,[자기관리],생각정리법,아카바 유지,"보유 5, 대출 3, 예약 0, 누적대출 25, 누적예약 5","책소개\n“망설이지 말고, 당황하지 말고\n0.1초 안에 생각해서 당당하게 반박하라..."
...,...,...,...,...,...
8615,[문학],고무보트를 타고 상어 잡는 법,모르텐 스트뢰크스네스,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",책소개\n“헤밍웨이의 《노인과 바다》 같은 고전이 될 만하다!”\n\n_〈베르겐스 ...
8616,[자기관리],지지 않는 대화,다카하시 켄타로,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0","책소개\n2,500년 동안 세계를 움직여온 무적의 대화법!\n\n아리스토텔레스의 《..."
8617,[문학],헤밍웨이 죽이기,엘러리 퀸,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",책소개\n“\n이보다 더 걸출한 작가들의 선집을 상상하기란 불가능하다!\n12명의 ...
8618,[자기관리],단순하게 생각하는 연습,구사나기 류슌,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0","책소개\n《반응하지 않는 연습》의 저자,\n구사나기 류슌이 소개하는\n내 마음 정리..."


### 정규표현식을 이용한 데이터 처리

In [22]:
import re

analysis_df = result_df
analysis_df

,kind,tit,writer,rent,0
0,[자기관리],그릿 GRIT,앤절라 더크워스,"보유 5, 대출 2, 예약 0, 누적대출 37, 누적예약 1","책소개\n*** TED 조회 수 1,000만 돌파! *** 전세계 25개국 동시 출..."
1,[문학],"베로니카, 죽기로 결심하다",파울로 코엘료,"보유 5, 대출 3, 예약 0, 누적대출 28, 누적예약 1","책소개\n세계적 밀리언셀러 작가, '소설의 연금술사'로 불리는 파울로 코엘료의 신작..."
2,[인문／사회],매우 예민한 사람들을 위한 책,전홍진,"보유 5, 대출 5, 예약 1, 누적대출 28, 누적예약 6",책소개\n예민성은 어떻게 조절될 수 있을까\n\n저자는 이 책에 나오는 사람마다 상...
3,[문학],작별하지 않는다,한강,"보유 5, 대출 3, 예약 0, 누적대출 27, 누적예약 0",책소개\n무엇을 생각하면 견딜 수 있나.\n가슴에 활활 일어나는 불이 없다면.\n기...
4,[자기관리],생각정리법,아카바 유지,"보유 5, 대출 3, 예약 0, 누적대출 25, 누적예약 5","책소개\n“망설이지 말고, 당황하지 말고\n0.1초 안에 생각해서 당당하게 반박하라..."
...,...,...,...,...,...
8615,[문학],고무보트를 타고 상어 잡는 법,모르텐 스트뢰크스네스,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",책소개\n“헤밍웨이의 《노인과 바다》 같은 고전이 될 만하다!”\n\n_〈베르겐스 ...
8616,[자기관리],지지 않는 대화,다카하시 켄타로,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0","책소개\n2,500년 동안 세계를 움직여온 무적의 대화법!\n\n아리스토텔레스의 《..."
8617,[문학],헤밍웨이 죽이기,엘러리 퀸,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",책소개\n“\n이보다 더 걸출한 작가들의 선집을 상상하기란 불가능하다!\n12명의 ...
8618,[자기관리],단순하게 생각하는 연습,구사나기 류슌,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0","책소개\n《반응하지 않는 연습》의 저자,\n구사나기 류슌이 소개하는\n내 마음 정리..."


In [23]:
# 특수문자 제거
analysis_df[0]=analysis_df[0].str.replace(pat=r'[^\w]', repl=r'', regex=True)
analysis_df[0]

0       책소개TED조회수1000만돌파전세계25개국동시출간아마존25주연속베스트셀러1위뉴욕타임...
1       책소개세계적밀리언셀러작가소설의연금술사로불리는파울로코엘료의신작소설베로니카죽기로결심하다...
2       책소개예민성은어떻게조절될수있을까저자는이책에나오는사람마다상황에맞게증상을설명하고그와관련...
3       책소개무엇을생각하면견딜수있나가슴에활활일어나는불이없다면기어이돌아가껴안을네가없다면이곳에...
4       책소개망설이지말고당황하지말고01초안에생각해서당당하게반박하라어떤상황에서도자기의견을확실...
                              ...                        
8615    책소개헤밍웨이의노인과바다같은고전이될만하다_베르겐스티덴데독창적인언어로엮어낸북유럽최고의...
8616    책소개2500년동안세계를움직여온무적의대화법아리스토텔레스의변론술은자신의주장을설득력있게...
8617    책소개이보다더걸출한작가들의선집을상상하기란불가능하다12명의노벨문학상과퓰리처상수상자들이...
8618    책소개반응하지않는연습의저자구사나기류슌이소개하는내마음정리정돈법마음을어떻게움직이고어떻게...
8619    책소개이책의탄생은그자체로놀라운사건이다학계가극찬한완성도높은과학그래픽노블의탄생유전자탄생...
Name: 0, Length: 8620, dtype: object

In [24]:
# 특정문자 제거

remove_list = ['책소개', '한줄','별점5개','별점4개','별점3개','별점2개',
               '별점1개','한글' '40자이내', '감사의글', '추천도서', '주석', '서평', '저자소개']

for i in remove_list:
    analysis_df[0]=analysis_df[0].str.replace(i,'')
analysis_df[0]

0       TED조회수1000만돌파전세계25개국동시출간아마존25주연속베스트셀러1위뉴욕타임스25...
1       세계적밀리언셀러작가소설의연금술사로불리는파울로코엘료의신작소설베로니카죽기로결심하다매마른...
2       예민성은어떻게조절될수있을까저자는이책에나오는사람마다상황에맞게증상을설명하고그와관련한연구...
3       무엇을생각하면견딜수있나가슴에활활일어나는불이없다면기어이돌아가껴안을네가없다면이곳에살았던...
4       망설이지말고당황하지말고01초안에생각해서당당하게반박하라어떤상황에서도자기의견을확실히말하...
                              ...                        
8615    헤밍웨이의노인과바다같은고전이될만하다_베르겐스티덴데독창적인언어로엮어낸북유럽최고의논픽션...
8616    2500년동안세계를움직여온무적의대화법아리스토텔레스의변론술은자신의주장을설득력있게전달하...
8617    이보다더걸출한작가들의선집을상상하기란불가능하다12명의노벨문학상과퓰리처상수상자들이남긴범...
8618    반응하지않는연습의저자구사나기류슌이소개하는내마음정리정돈법마음을어떻게움직이고어떻게생각해...
8619    이책의탄생은그자체로놀라운사건이다학계가극찬한완성도높은과학그래픽노블의탄생유전자탄생이후의...
Name: 0, Length: 8620, dtype: object

In [25]:
analysis_df

,kind,tit,writer,rent,0
0,[자기관리],그릿 GRIT,앤절라 더크워스,"보유 5, 대출 2, 예약 0, 누적대출 37, 누적예약 1",TED조회수1000만돌파전세계25개국동시출간아마존25주연속베스트셀러1위뉴욕타임스25...
1,[문학],"베로니카, 죽기로 결심하다",파울로 코엘료,"보유 5, 대출 3, 예약 0, 누적대출 28, 누적예약 1",세계적밀리언셀러작가소설의연금술사로불리는파울로코엘료의신작소설베로니카죽기로결심하다매마른...
2,[인문／사회],매우 예민한 사람들을 위한 책,전홍진,"보유 5, 대출 5, 예약 1, 누적대출 28, 누적예약 6",예민성은어떻게조절될수있을까저자는이책에나오는사람마다상황에맞게증상을설명하고그와관련한연구...
3,[문학],작별하지 않는다,한강,"보유 5, 대출 3, 예약 0, 누적대출 27, 누적예약 0",무엇을생각하면견딜수있나가슴에활활일어나는불이없다면기어이돌아가껴안을네가없다면이곳에살았던...
4,[자기관리],생각정리법,아카바 유지,"보유 5, 대출 3, 예약 0, 누적대출 25, 누적예약 5",망설이지말고당황하지말고01초안에생각해서당당하게반박하라어떤상황에서도자기의견을확실히말하...
...,...,...,...,...,...
8615,[문학],고무보트를 타고 상어 잡는 법,모르텐 스트뢰크스네스,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",헤밍웨이의노인과바다같은고전이될만하다_베르겐스티덴데독창적인언어로엮어낸북유럽최고의논픽션...
8616,[자기관리],지지 않는 대화,다카하시 켄타로,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",2500년동안세계를움직여온무적의대화법아리스토텔레스의변론술은자신의주장을설득력있게전달하...
8617,[문학],헤밍웨이 죽이기,엘러리 퀸,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",이보다더걸출한작가들의선집을상상하기란불가능하다12명의노벨문학상과퓰리처상수상자들이남긴범...
8618,[자기관리],단순하게 생각하는 연습,구사나기 류슌,"보유 2, 대출 0, 예약 0, 누적대출 0, 누적예약 0",반응하지않는연습의저자구사나기류슌이소개하는내마음정리정돈법마음을어떻게움직이고어떻게생각해...


In [26]:
# 저장하기
analysis_df.to_csv("세미 프로젝트 책추천 크롤링.csv", encoding='utf-8-sig')